In [1]:
#import libraries
import functions as func
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

#load preprocessed data
df = func.getUseableHP2Data()

In [ ]:
# TODO: fix na situation for houseprices 2
df = pd.read_csv("houseprices2.csv")

pool = df["PoolQC"].tolist()
fence = df["Fence"].tolist()
misc = df["MiscFeature"].tolist()

pool2 = []
fence2 = []
misc2 = []
for i in range(len(df)):
    try:
        if np.isnan(pool[i]):
            pool2.append("NotApplicable")
        else:
            pool2.append(pool[i])
    except:
        pool2.append(pool[i])

    try:
        if np.isnan(fence[i]):
            fence2.append("NotApplicable")
        else:
            fence2.append(fence[i])
    except:
        fence2.append(fence[i])

    try:
        if np.isnan(misc[i]):
            misc2.append("NotApplicable")
        else:
            misc2.append(misc[i])
    except:
        misc2.append(misc[i])


df["PoolQC"] = pool2
df["Fence"] = fence2
df["MiscFeature"] = misc2

print(len(df))
df.dropna(inplace=True)
print(len(df))

Baseline model

In [ ]:
#fit and score full baseline model
X = df.copy(deep=True)
X.drop(["SalePrice"], axis=1, inplace=True)
y = df["SalePrice"]
base_mae = func.trainLinearRegression(X, y)
print(base_mae)

Variable selection model

In [ ]:
#get first new predictor
colnames = list(df.columns)
colnames.remove("SalePrice")
colnames = [[x] for x in colnames]
results = Parallel(n_jobs=6)(delayed(func.quickScoreLinearRegression)(i, True) for i in colnames)
first_new_colname = colnames[np.argmax(results)]
print(f"Therefore we now add {first_new_colname} as a model feature")

In [ ]:
#get second new predictor
colnames2 = list(df.columns)
colnames2.remove("SalePrice")
colnames2.remove(first_new_colname)
collnames2 = [[x, first_new_colname] for x in colnames2]
results = Parallel(n_jobs=6)(delayed(func.quickScoreLinearRegression)(i, True) for i in colnames2)
second_new_colname = colnames2[np.argmax(results)]
print(f"Therefore we now add {second_new_colname} as a model feature")

In [ ]:
#get third and final new predictor
colnames3 = list(df.columns)
colnames3.remove("SalePrice")
colnames3.remove(first_new_colname)
colnames3.remove(second_new_colname)
colnames3 = [[x, first_new_colname, second_new_colname] for x in colnames3]
results = Parallel(n_jobs=6)(delayed(func.quickScoreLinearRegression)(i, True) for i in colnames)
third_new_colname = colnames3[np.argmax(results)]
print(f"Finally we add {third_new_colname} as a model feature")
print(f"This gives our model a MAE of {min(results)}.")

Random forest regressor

In [2]:
rf_mae = func.quickScoreRandomForest()
print(rf_mae)

35187.0
